This is the second assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Again, please insert to code to your ApacheCouchDB based Cloudant instance below using the "Insert Code" function of Watson Studio( you've done this in Assignment 1 before)

Done, just execute all cells one after the other and you are done - just note that in the last one you have to update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"

In [1]:
#your cloudant credentials go here
###YOUR_CODE_GOES_HERE###"



Let's create a SparkSession object and put the Cloudant credentials into it

In [2]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .config("jsonstore.rdd.partitions", 1)\
    .getOrCreate()

Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [3]:
df=spark.read.load('shake_classification', "com.cloudant.spark")

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+--------+-----+-----+-----+--------------------+--------------------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|
+-----+--------+-----+-----+-----+--------------------+--------------------+
|    1|aaaaaaaa|-0.02|-0.02|-0.02|175de817e9ba9c70b...|1-2a4c03e5d983364...|
|    1|aaaaaaaa|-0.01|-0.01|-0.01|175de817e9ba9c70b...|1-aedd08521768e86...|
|    1|aaaaaaaa|-0.02|-0.02|-0.02|175de817e9ba9c70b...|1-2a4c03e5d983364...|
|    1|aaaaaaaa|-0.01|-0.01|-0.01|175de817e9ba9c70b...|1-aedd08521768e86...|
|    1|aaaaaaaa|-0.02|-0.02|-0.02|175de817e9ba9c70b...|1-2a4c03e5d983364...|
|    1|aaaaaaaa| 0.01| 0.01| 0.01|175de817e9ba9c70b...|1-50911026f3d1209...|
|    1|aaaaaaaa| 0.01| 0.01| 0.01|175de817e9ba9c70b...|1-50911026f3d1209...|
|    1|aaaaaaaa| 0.19| 0.19| 0.19|175de817e9ba9c70b...|1-fb4ef4cb99a62a5...|
|    1|aaaaaaaa| 0.01| 0.01| 0.01|175de817e9ba9c70b...|1-50911026f3d1209...|
|    1|aaaaaaaa|-0.02|-0.02|-0.02|175de817e9ba9c70b...|1-2a4c03e5d983364...|

Please create a VectorAssembler which consumed columns X, Y and Z and produces a column “features”


In [26]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = ###YOUR_CODE_GOES_HERE###"

Please insatiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to either
1.	Rename the “CLASS” column to “label” or
2.	Specify the label-column correctly to be “CLASS”


In [27]:
from pyspark.ml.classification import ###YOUR_CODE_GOES_HERE###"

classifier = ###YOUR_CODE_GOES_HERE###"



Let’s train and evaluate…


In [28]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [29]:
model = pipeline.fit(df)

In [31]:
prediction = model.transform(df)

In [35]:
prediction.show()

+-----+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|label|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|           features|prediction|
+-----+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|    0|    1|aaaaaaaa|-0.02|-0.02|-0.02|175de817e9ba9c70b...|1-2a4c03e5d983364...|[-0.02,-0.02,-0.02]|       0.0|
|    0|    1|aaaaaaaa|-0.01|-0.01|-0.01|175de817e9ba9c70b...|1-aedd08521768e86...|[-0.01,-0.01,-0.01]|       0.0|
|    0|    1|aaaaaaaa|-0.02|-0.02|-0.02|175de817e9ba9c70b...|1-2a4c03e5d983364...|[-0.02,-0.02,-0.02]|       0.0|
|    0|    1|aaaaaaaa|-0.01|-0.01|-0.01|175de817e9ba9c70b...|1-aedd08521768e86...|[-0.01,-0.01,-0.01]|       0.0|
|    0|    1|aaaaaaaa|-0.02|-0.02|-0.02|175de817e9ba9c70b...|1-2a4c03e5d983364...|[-0.02,-0.02,-0.02]|       0.0|
|    0|    1|aaaaaaaa| 0.01| 0.01| 0.01|175de817e9ba9c70b...|1-50911026f3d1209...|   [0.

In [32]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("label")
    
binEval.evaluate(prediction) 

0.631578947368421

If you are happy with the result (I’m happy with > 0.55) please submit your solution to the grader by executing the following cells, please don’t forget to obtain an assignment submission token (secret) from the Courera’s graders web page and paste it to the “secret” variable below, including your email address you’ve used for Coursera. (0.55 means that you are performing better than random guesses)


In [36]:
!rm -Rf a2_m2.json

In [37]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m2.json')

In [ ]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

In [39]:
!zip -r a2_m2.json.zip a2_m2.json

  adding: a2_m2.parquet/ (stored 0%)
  adding: a2_m2.parquet/.part-00000-1ff92a81-68c8-446b-a54e-a042a8fd7f1e.snappy.parquet.crc (stored 0%)
  adding: a2_m2.parquet/part-00000-1ff92a81-68c8-446b-a54e-a042a8fd7f1e.snappy.parquet (deflated 44%)
  adding: a2_m2.parquet/_SUCCESS (stored 0%)
  adding: a2_m2.parquet/._SUCCESS.crc (stored 0%)


In [41]:
!base64 a2_m2.json.zip > a2_m2.json.zip.base64

In [1]:
from rklib import submit
key = "J3sDL2J8EeiaXhILFWw2-g"
part = "G4P6f"
email = ###YOUR_CODE_GOES_HERE###"
secret = ###YOUR_CODE_GOES_HERE###"

with open('a2_m2.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
